<a href="https://colab.research.google.com/github/Zamoca42/TIL/blob/main/Python/Pyside_widget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch02. Pyside 위젯2

## 메신저 만들어보기

### ui design

<img width="399" alt="스크린샷 2023-01-25 오후 5 52 19" src="https://user-images.githubusercontent.com/96982072/214583838-8217fa89-74d9-4027-a081-a062f7abad1b.png">

- list view box
  - object name : `list_chat`

- line edit(입력 박스)
  - nickname_object : `edit_nickname`
  - chat_object : `edit_text`

- push button
  - object name : `btn_send`

### 실습코드

1. 위의 ui를 python 파일로 변환  
  ```
  pyside6-uic ui.ui > ui.py
  ```
2. `edit_text`에 내용 입력 시 메세지 박스로 출력

In [ ]:
#...

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.ui.btn_send.clicked.connect(self.send)
    
    def send(self):
        mb = QMessageBox()
        mb.setText(self.ui.edit_text.text())
        mb.exec()

#...

- 결과  

  ![스크린샷 2023-01-25 오후 7 37 38](https://user-images.githubusercontent.com/96982072/214542052-4952ea7c-7d60-4a29-a539-eb09a770586d.png)

### List View

- List View
  - model이 포함되지 않음
    - item과 model을 설정해주어야함

- Model 
  - 어플리케이션에서 사용되는 데이터와 그 데이터를 처리하는 부분
- View 
  - 사용자에서 보여지는 UI 부분
- Controller 
  - 사용자의 입력(Action)을 받고 처리하는 부분

- MVC, MVVM 패턴
  - https://beomy.tistory.com/43


### 실습코드2

1. `list_chat`에서 setModel
2. text를 입력하고 전송을 누르면 list_view에서 보여줌

In [ ]:
import sys
from PySide6.QtWidgets import QMainWindow, QApplication, QMessageBox
from ui import Ui_MainWindow # ui.py

from PySide6.QtGui import QStandardItemModel, QStandardItem

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.model = QStandardItemModel()
        self.ui.list_chat.setModel(self.model)

        self.ui.btn_send.clicked.connect(self.send)
    
    def send(self):
        text = self.ui.edit_text.text()
        item = QStandardItem(text)
        self.model.appendRow(item)

#...

- 결과  
  
  ![스크린샷 2023-01-25 오후 7 51 54](https://user-images.githubusercontent.com/96982072/214550593-36cc1c4a-cc81-4f0a-a74b-9955fbce32fa.png)


### List widget, line edit event

- list_view -> list widget으로 변경  

  <img width="529" alt="스크린샷 2023-01-25 오후 10 37 06" src="https://user-images.githubusercontent.com/96982072/214589519-b1484341-f835-4216-823b-ae8c74b370e8.png">

  - list widget은 model이 포함
    - 설정할 필요 없음

### 실습코드3


- list_view -> list widget으로 변경
- nickname 추가
- 전송버튼 대신 enter키 누르면 전송기능
  - `returnPressed.connect(self.send)`

In [ ]:
class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        #self.model = QStandardItemModel()
        #self.ui.list_chat.setModel(self.model)

        self.ui.btn_send.clicked.connect(self.send)
        self.ui.edit_text.returnPressed.connect(self.send)
    
    def send(self):
        nickname = self.ui.edit_nickname.text()
        text = self.ui.edit_text.text()
        # item = QStandardItem(text)
        # self.model.appendRow(item)
        self.ui.list_chat.addItem(f"{nickname}: {text}")
#...

- 결과  
  
  ![스크린샷 2023-01-25 오후 9 52 54](https://user-images.githubusercontent.com/96982072/214588253-5563b9d7-71a3-4391-88fd-077635d5e549.png)


## file

### 실습 코드

- 메신저을 위한 브로드캐스트 서버대신 파일을 사용
- 랜덤닉네임 생성해서 불러오기
- 채팅 내용을 전송후에는 비우기


In [ ]:
#...

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        #self.model = QStandardItemModel()
        #self.ui.list_chat.setModel(self.model)

        self.ui.btn_send.clicked.connect(self.send)
        self.ui.edit_text.returnPressed.connect(self.send)

        nickname = self.random_nickname()
        self.ui.edit_nickname.setText(nickname)
    
    def send(self):
        nickname = self.ui.edit_nickname.text()
        text = self.ui.edit_text.text()
        # item = QStandardItem(text)
        # self.model.appendRow(item)
        # self.ui.list_chat.addItem(f"{nickname}: {text}")
        msg = f"{nickname}: {text}"

        # 파일에다가 msg 쓰기
        with open("./server.txt", "a+") as f:
            f.writelines(msg)
        
        self.ui.edit_text.clear()
    
    def random_nickname(self):
        nickname = random.choice(["홍길동", "박보검", "한소희"])
        num = random.randint(1,1000)
        return f"{nickname}{num}"

#...

- 결과  
  
  ![스크린샷 2023-01-25 오후 10 14 40](https://user-images.githubusercontent.com/96982072/214594096-b24326d9-742d-4f11-9e01-5843f1695ac1.png)




### 여러명 채팅

- 창 여러개를 띄워 여러명과 채팅하는 상황 만들기
- 서버 파일 로드해보기
  - listen
  - server.txt에 저장된 메세지 리스트 박스로 불러오기
  - 한번 불러온 메세지는 다시 불러오지 않기
  - 처음 실행할 때 저장된 메세지 모두 불러오기

- 타이머 만들고 특정 시간마다 server.txt 불러오기
  - `QTimer()`

- 처음 실행 시 환영 메세지 추가

In [ ]:
from PySide6.QtCore import QTimer

# 타이머
self.timer = QTimer()
self.timer.setInterval(500)
self.timer.timeout.connect(self.listen)
self.timer.start()

### 실습코드

In [ ]:
class MainWindow(QMainWindow):
    last_read = 0

    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.ui.btn_send.clicked.connect(self.send)
        self.ui.edit_text.returnPressed.connect(self.send)

        nickname = self.random_nickname()
        self.ui.edit_nickname.setText(nickname)

        # 환영합니다 메세지
        with open("./server.txt", "a+") as f:
            f.writelines(f"---------{nickname}님이 입장하셨습니다\n")

        self.listen()

        self.timer = QTimer()
        self.timer.setInterval(500)
        self.timer.timeout.connect(self.listen)
        self.timer.start()
        
    def send(self):
        nickname = self.ui.edit_nickname.text()
        text = self.ui.edit_text.text()
        msg = f"{nickname}: {text}"

        # 파일에다가 msg 쓰기
        with open("./server.txt", "a+") as f:
            f.writelines(msg + "\n")
        
        self.ui.edit_text.clear()

        # # 읽어오기
        # self.listen()
    
    def random_nickname(self):
        nickname = random.choice(["홍길동", "박보검", "한소희"])
        num = random.randint(1,1000)
        return f"{nickname}{num}"
    
    def listen(self):
        try:
            with open("./server.txt", "r") as f:
                lines = f.readlines()
            lines = [x.strip() for x in lines]
            self.ui.list_chat.addItems(lines[self.last_read:])
            self.last_read = len(lines)
            self.ui.list_chat.scrollToBottom()
        except:
            pass

- 결과

  ![스크린샷 2023-01-26 오전 1 15 28](https://user-images.githubusercontent.com/96982072/214617579-6a9d503f-c695-4a92-b709-5f02cc78f1e3.png)
